In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
%matplotlib inline

#train_final.csv - the training set of interactions
train_final = pd.read_csv('input/train_final.csv', delimiter = "\t");

#tracks_final.csv - supplementary information about the items
tracks_final = pd.read_csv('input/tracks_final.csv', delimiter = "\t");

#playlists_final.csv - supplementary information about the users
playlists_final = pd.read_csv('input/playlists_final.csv', delimiter = "\t");

#target_playlists.csv - the set of target playlists that will receive recommendations
target_playlists = pd.read_csv('input/target_playlists.csv');

#target_tracks.csv - the set of target items (tracks) to be recommended
target_tracks = pd.read_csv('input/target_tracks.csv');

#Let's have a look at the train data. 
train_final.head()

,playlist_id,track_id
0,3271849,2801526
1,5616275,727878
2,11267488,2805283
3,10103900,1515105
4,3836898,2945623


In [3]:
target_tracks.shape[0]

32195

In [41]:
item_playlist_matrix = np.zeros([playlists_final.shape[0], tracks_final.shape[0]]) 

In [43]:
#The matrix has playlists as rows and items as columns
item_playlist_matrix.size

5756100000

In [52]:
#This is too big.. We need to remove some items! 

#Lets remove all tracks whith less than 10 in popularity, and add the rest to a set called relevant_tracks
#Popularity is now a dataframe
popularity = train_final.groupby(by="track_id").playlist_id.nunique().to_frame()

#remove index name
popularity.reset_index(level = 0, inplace = True)

#Rename the columns
popularity.columns = ['track_id','occurrences']

In [123]:
#filtering all tracks occurring less than 10 times gives 23298 rows

#filtering all tracks occurring less than 20 times gives 9489 rows

tracks_relevant = popularity[popularity['occurrences'] > 20]

In [124]:
item_playlist_matrix = np.zeros([playlists_final.shape[0], tracks_relevant.shape[0]]) 

In [125]:
#This saves us a factor of 10, but we still have a huge matrix. 
item_playlist_matrix.shape

(57561, 9489)

In [126]:
# Now we should be able to filter away interactions with all tracks that we removed. 
train_relevant = train_final[train_final['track_id'].isin(tracks_relevant['track_id'])]
train_relevant.describe()

,playlist_id,track_id
count,4.368620e+05,4.368620e+05
mean,6.530891e+06,1.992278e+06
std,3.079244e+06,1.141203e+06
min,7.569000e+03,1.450000e+03
25%,4.184420e+06,1.056176e+06
50%,6.663308e+06,2.020026e+06
75%,8.691742e+06,2.924245e+06
max,1.176636e+07,4.936513e+06


In [127]:
#Let's not throw away playlists!  see if we can throw away some playlists. 

#playlist_occurrences = train_relevant.groupby(by="playlist_id").track_id.nunique().to_frame()

#remove index name
#playlist_occurrences.reset_index(level = 0, inplace = True)

#Rename the columns
#playlist_occurrences.columns = ['playlist_id','occurrences']

In [128]:
#Playlists with more than 5 songs: 17173

#playlists_relevant = playlist_occurrences[playlist_occurrences['occurrences'] > 5]
#playlists_relevant.describe()

In [131]:
#Let's try again.  

#We have to add +1 because row/col 0 will be taken. 
item_playlist_matrix = np.zeros([playlists_final.shape[0]+1, tracks_relevant.shape[0]+1]) 
print(item_playlist_matrix.size)
print(item_playlist_matrix.shape)

546263380
(57562, 9490)


In [135]:
#Now, we should filter the train_final so it only contains relevant stuff.

playlists_relevant = pd.concat([playlists_relevant, target_playlists])

train_relevant = train_final[train_final['playlist_id'].isin(playlists_relevant['playlist_id'])]

#Easy. Now it is down to 436862 interactions, should not be too slow. 
#train_relevant.head()

train_relevant.describe()

,playlist_id,track_id
count,8.826700e+05,8.826700e+05
mean,6.526798e+06,1.987094e+06
std,3.086458e+06,1.136636e+06
min,7.614000e+03,2.520000e+02
25%,4.181885e+06,1.053281e+06
50%,6.647196e+06,2.015358e+06
75%,8.693782e+06,2.911595e+06
max,1.176636e+07,5.018274e+06


In [147]:
#If we translate each track_id to a track_index which will serve as matrix index, we can save a lot of time. 
#Same goes for playlist_id --> playlist_index. 


#We need a way to get from track_id to index in O(1).
#Let's create a dictionary

track_indexes = {}
counter = 1; #Index 0 is reserved for playlist_ids
for track_id in tracks_relevant['track_id']:
    track_indexes[track_id] = counter
    counter += 1;
    
#and a way to get from playlist_id to index in O(1)

playlists_relevant

playlist_indexes = {}
counter = 1; #Index 0 is reserved for track_ids
for playlist_id in playlists_relevant['playlist_id']:
    playlist_indexes[playlist_id] = counter
    counter += 1;

#felsökning
print(playlists_relevant[playlists_relevant['playlist_id']==1515105])

#Now, in order to get a playlist_index we just go: playlist_index = playlist_indexes[playlist_id]
    
#How do we get it back in the end? We simply keep the tracks_relevant and playlist_relevant and access them by index in the end. 
# tracks_relevant[playlist_index] = playlist_id

Empty DataFrame
Columns: [occurrences, playlist_id]
Index: []


In [143]:
train_relevant = train_final[train_final['playlist_id'].isin(playlists_relevant['playlist_id'])]
train_relevant.shape

(882670, 2)

In [144]:
#Lets build that matrix. 

interactions = train_relevant.as_matrix()
for row in interactions:
    #Lets get the info
    playlist_id = row[0]
    track_id = row[1]
    
    #Now lets get the proper indexes. 
    playlist_index = playlist_indexes[playlist_id]
    track_index = track_indexes[track_id]
    
    #And now lets add it to the matrix
    item_playlist_matrix[playlist_index][track_index] = 1
    

item_playlist_matrix
    


KeyError: 1515105